In [28]:
import sys 
import sys 
from keyvars import ufiles_path
sys.path.append(ufiles_path)

import uvars
import uviz
import uerrortab as uterror
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
paths = uvars.parqts

In [2]:
paths = uvars.parqts
paths

['/mnt/c/Users/Joseph/Documents/phd/datasets/TILEex/idata/sample_patches_parqt/sample_patches_parqt_9idx.parquet',
 '/mnt/c/Users/Joseph/Documents/phd/datasets/TILEex/idata/sample_patches_parqt/sample_patches_parqt_15idx.parquet',
 '/mnt/c/Users/Joseph/Documents/phd/datasets/TILEex/idata/sample_patches_parqt/sample_patches_parqt_25idx.parquet',
 '/mnt/c/Users/Joseph/Documents/phd/datasets/TILEex/idata/sample_patches_parqt/sample_patches_parqt_300idx.parquet']

In [3]:
df = pd.read_parquet(paths[0])

In [19]:
demcols = ['cop', 'edem', 'pband','tdemx']
ftcols = ['cop', 'edem', 'pband', 's1', 'tdemx', 'wc', 'wc_ffth',
       'wc_gau', 'wc_sobelm', 'wc_sobelh', 'wc_sobelv']

fcolx = ['tdemx','edem', 'wc_ffth','wc_gau', 'wc_sobelm', 'wc_sobelh', 'wc_sobelv']
tcolx = 'pband'

In [20]:
#: use my iter cb pipeline

In [79]:
from sklearn.ensemble import (
    RandomForestRegressor,GradientBoostingRegressor, 
    VotingRegressor,StackingRegressor)

from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
from pprint import pprint

In [80]:
models = [
    ('rf',RandomForestRegressor()),
    ('gb',GradientBoostingRegressor()),
    ('br',BayesianRidge())
]

weights = [0.7, 0.9, 0.5]

In [23]:
ensemble = VotingRegressor(estimators=models, weights=weights)

In [25]:
X = df[fcolx].values 
y = df[tcolx].values

X.shape, y.shape

((65536, 7), (65536,))

In [59]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.30, random_state=1)
# split the full train set into train and validation sets
xtrain, xvalid, ytrain, yvalid = train_test_split(Xtrain, Ytrain, test_size=0.33, random_state=1)

print(f'train: {xtrain.shape} {ytrain.shape}')
print(f'valid: {xvalid.shape} {yvalid.shape}')
print(f'test :{Xtrain.shape} {Ytrain.shape}')

train: (30736, 7) (30736,)
valid: (15139, 7) (15139,)
test :(45875, 7) (45875,)


In [86]:
def get_models():
    models.append()
    models.append(('rf',RandomForestRegressor()))
    models.append(('gb',GradientBoostingRegressor()))
    models.append(('br',BayesianRidge()))
    return models

def transform_rmse_to_percentage(rmse, max_rmse):
    percentage_rmse = (1 - rmse / max_rmse) #* 100
    return percentage_rmse

In [87]:
def evaluate_models(models, xtrain, xvalid, ytrain, yvalid, T=False):
    scores = []
    for name, model in models:
        model.fit(xtrain,ytrain)
        pvalid = model.predict(xvalid)
        error = uterror.get_tab_metrics(yvalid, pvalid)
        #print(error)
        R2,ME, RMSE, MAD, LE90, EVS = error
        ptc = -RMSE
        if T:
            ptc = transform_rmse_to_percentage(RMSE, 100)
        
        print(f'{name} {RMSE} {ptc}')
        # define accuracy metric for regression that does not go banana like R2
      
        scores.append(ptc)
    return scores

def evaluate_ensemble(ensemble, x1,x2, y1,y2):
    ensemble.fit(x1,y1)
    p2 = ensemble.predict(x2)
    error = uterror.get_tab_metrics(y2, p2)
    R2,ME, RMSE, MAD, LE90, EVS = error
    ptc = transform_rmse_to_percentage(RMSE, 100)
    print(f'RMSE:{RMSE} {ptc}% \nR2:{R2} EVS:{EVS} MBE:{ME} MAD:{MAD} L90:{LE90}')
    #if pct:
        #ptc = transform_rmse_to_percentage(RMSE, 100)
        #print(f'RMSE val:{RMSE} pct:{ptc}')

In [88]:
def context(text):
    print('---'*50)
    print(f'{text}')
    print('---'*50)
    print('')

In [89]:
context('Using Scores RMSE values Transformed ') # try MAE or my PERC
scores = evaluate_models(models, xtrain, xvalid, ytrain, yvalid, True)
context('model scores as weights')
wensemble = VotingRegressor(estimators=models,weights=scores)
mensemble = VotingRegressor(estimators=models)
context('wensemble')
evaluate_ensemble(wensemble,Xtrain, Xtest, Ytrain, Ytest)
context('mensemble')
evaluate_ensemble(mensemble,Xtrain, Xtest, Ytrain, Ytest)

------------------------------------------------------------------------------------------------------------------------------------------------------
Using Scores RMSE values Transformed 
------------------------------------------------------------------------------------------------------------------------------------------------------

rf 1.2825887529641644 0.9871741124703584
gb 1.4972440834739704 0.9850275591652603
br 2.590048627323648 0.9740995137267635
------------------------------------------------------------------------------------------------------------------------------------------------------
model scores as weights
------------------------------------------------------------------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------------------------------------------------
wensemble
-------------------------------------------------

In [91]:
context('Using Scores RMSE values') # try MAE or my PERC
scores = evaluate_models(models, xtrain, xvalid, ytrain, yvalid, False)
context('model scores as weights')
wensemble = VotingRegressor(estimators=models,weights=scores)
mensemble = VotingRegressor(estimators=models)
context('wensemble')
evaluate_ensemble(wensemble,Xtrain, Xtest, Ytrain, Ytest)
context('mensemble')
evaluate_ensemble(mensemble,Xtrain, Xtest, Ytrain, Ytest)

------------------------------------------------------------------------------------------------------------------------------------------------------
Using Scores RMSE values
------------------------------------------------------------------------------------------------------------------------------------------------------

rf 1.283108930664768 -1.283108930664768
gb 1.4972457874684382 -1.4972457874684382
br 2.590048627323648 -2.590048627323648
------------------------------------------------------------------------------------------------------------------------------------------------------
model scores as weights
------------------------------------------------------------------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------------------------------------------------
wensemble
--------------------------------------------------------------

We expect the ensemble to perform better than any contributing ensemble member, and this can be checked directly by evaluating each member model on the full train and test sets independently.

In [92]:
# isnt that because we train the enbsemles in the full data alone

In [93]:
scores_f = evaluate_models(models,Xtrain, Xtest, Ytrain, Ytest, False)

rf 1.2732282826105201 -1.2732282826105201
gb 1.5024498266047015 -1.5024498266047015
br 2.5714964366190096 -2.5714964366190096


In [94]:
# RF outperfoms both ensemble, but both esembles out performs other models

In [95]:
context('Using Scores RMSE values') # try MAE or my PERC
scores = evaluate_models(models, xtrain, xvalid, ytrain, yvalid, False)

print(scores)
ranking = 1 + np.argsort(np.argsort(scores))
print(ranking)

context('model scores as weights')
wkensemble = VotingRegressor(estimators=models,weights=ranking)
mensemble = VotingRegressor(estimators=models)
context('wensemble')
evaluate_ensemble(wensemble,Xtrain, Xtest, Ytrain, Ytest)
context('mensemble')
evaluate_ensemble(mensemble,Xtrain, Xtest, Ytrain, Ytest)
context('each model scores')
scores_f = evaluate_models(models,Xtrain, Xtest, Ytrain, Ytest, False)

------------------------------------------------------------------------------------------------------------------------------------------------------
Using Scores RMSE values
------------------------------------------------------------------------------------------------------------------------------------------------------

rf 1.2869847217666575 -1.2869847217666575
gb 1.4972322890656196 -1.4972322890656196
br 2.590048627323648 -2.590048627323648
[-1.2869847217666575, -1.4972322890656196, -2.590048627323648]
[3 2 1]
------------------------------------------------------------------------------------------------------------------------------------------------------
model scores as weights
------------------------------------------------------------------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------------------------------------------------

source:
> https://machinelearningmastery.com/weighted-average-ensemble-with-python/

In [76]:
#

RMSE:1.739531389747936 0.9826046861025206% 
R2:0.8182774284319296 EVS:0.8182894546598853 MBE:1.329940634926246 MAD:1.0393677889031743 L90:2.808140673169685


RMSE:1.5478734267686194 0.9845212657323138% 
R2:0.8561150987684715 EVS:0.8561236402885903 MBE:1.1472321240396923 MAD:0.8760237121361278 L90:2.421826616278281


In [52]:
# train_test_split : violets the spatial assumption , it is random